In [1]:
from wrapper_functions import *

2022-10-21 13:10:03.653427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/gpflow/experimental/utils.py:43: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  f"You're calling {name} which is considered *experimental*."
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/gpflow/experimental/utils.py:43: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  f"You're calling {name} which i

TensorFlow version: 2.10.0. Expected: 2.7.0
TensorFlow Probability version: 0.18.0. Expected: 0.15.0


2022-10-21 13:10:26.159966: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Experiment II: 2D Spatial Field

### Model Configs

In [2]:
# Optimization configs. 
# Consider reduce below parameters / set to `False` if MCMC is taking too long:
# mcmc_num_steps, mcmc_burnin, mcmc_nchain, mcmc_initialize_from_map.
map_step_size=5e-4   # @param
map_num_steps=10_000  # @param

mcmc_step_size=1e-4 # @param
mcmc_num_steps=1000 # @param

mcmc_nchain=10 # @param
mcmc_burnin=2_500 # @param
bne_mcmc_initialize_from_map="True" # @param ["False", "True"]

bne_mcmc_initialize_from_map = eval(bne_mcmc_initialize_from_map)


In [3]:
# BMA parameters.
y_noise_std = 0.01  # Note: Changed from 0.1 # @param
bma_gp_lengthscale = .05 # @param
bma_gp_l2_regularizer = 0.1 # @param

bma_n_samples_train = 100 # @param
bma_n_samples_eval = 250 # @param
bma_n_samples_test = 250 # @param
bma_seed = 0 # @param


In [4]:
# BNE parameters.
bne_gp_lengthscale = .05 # 5. # @param
bne_gp_l2_regularizer = 1. # 15 # @param
bne_variance_prior_mean = -2.5 # @param
bne_skewness_prior_mean = -2.5 # @param
bne_seed = 0 # @param

### Read training/prediction data

In [5]:
training_eastMA = pd.read_csv('./data/training_dataset/training_eastMA.csv')
training_eastMA_noMI = training_eastMA[:51]
training_eastMA_folds = pd.read_csv('./data/training_dataset/training_eastMA_folds.csv')
base_model_predictions_eastMA = pd.read_csv('./data/prediction_dataset/base_model_predictions_eastMA.csv')
display(training_eastMA.shape, training_eastMA_folds.shape, base_model_predictions_eastMA.shape)
print("pred longitude max and min", base_model_predictions_eastMA["lon"].max(),base_model_predictions_eastMA["lon"].min())
print("pred latitude max and min", base_model_predictions_eastMA["lat"].max(),base_model_predictions_eastMA["lat"].min())
#list(base_model_predictions_eastMA.columns)
print("train longitude max and min", training_eastMA["lon"].max(),training_eastMA["lon"].min())
print("train latitude max and min", training_eastMA["lat"].max(),training_eastMA["lat"].min())


training51= pd.read_csv('./data/training_dataset/training51.csv')

(55, 7)

(55, 10)

(84421, 5)

pred longitude max and min -69.93 -73.5
pred latitude max and min 44.3 40.6
train longitude max and min -70.023598 -73.443056
train latitude max and min 44.107524 40.74529


In [6]:
# standardize
X_train1 = np.asarray(training_eastMA_noMI[["lon", "lat"]].values.tolist()).astype(np.float32)
X_test1 = np.asarray(base_model_predictions_eastMA[["lon", "lat"]].values.tolist()).astype(np.float32)
X_valid = np.concatenate((X_train1, X_test1), axis=0)
X_centr = np.mean(X_valid, axis=0)
X_scale = np.max(X_valid, axis=0) - np.min(X_valid, axis=0)

X_train1 = (X_train1 - X_centr) / X_scale
X_test1 = (X_test1 - X_centr) / X_scale

Y_train = np.expand_dims(training_eastMA_noMI["aqs"], 1).astype(np.float32)
#Y_test = np.expand_dims(base_model_predictions_eastMA["pred_av"], 1).astype(np.float32)

print("2011 center and scale: ", X_centr, X_scale)

2011 center and scale:  [-72.185104  42.680347] [3.5699997 3.7000008]


In [7]:
base_model_names = ["pred_av", "pred_gs", "pred_caces"]
base_preds_train = tf.stack([training_eastMA_noMI[base_model_name].astype(np.float32) for base_model_name in base_model_names], axis=-1)
base_preds_test = tf.stack([base_model_predictions_eastMA[base_model_name].astype(np.float32) for base_model_name in base_model_names], axis=-1)
#base_preds_test
display(base_preds_train.shape, base_preds_test.shape)

TensorShape([51, 3])

TensorShape([84421, 3])

## Bayesian Model Averaging

A Bayesian ensemble model where ensemble weights $w_k's$ are parameterized by Gaussian process priors:

$y \sim N(\mu(x), \sigma^2)$ 

$\mu(x) = \sum_{k=1}^K w_k(x) * m_k(x) \quad$  where $\{m_k\}_{k=1}^K$ are base model predictions.

$w(x) = softmax(f(x)) \qquad\;\;\;$ where $w=[w_1, \dots, w_K]$ and $f=[f_1, \dots, f_K]$

$f \stackrel{i.i.d.}{\sim} GaussianProcess(0, k)$




In [8]:
# Assemble into configs.
bma_model_config = DEFAULT_GP_CONFIG.copy()
map_config = DEFAULT_MAP_CONFIG.copy()
mcmc_config = DEFAULT_MCMC_CONFIG.copy()

bma_model_config.update(dict(lengthscale=bma_gp_lengthscale,
                             l2_regularizer=bma_gp_l2_regularizer,
                             y_noise_std=y_noise_std))

map_config.update(dict(learning_rate=map_step_size,
                       num_steps=map_num_steps))

mcmc_config.update(dict(step_size=mcmc_step_size, 
                        num_steps=mcmc_num_steps,
                       burnin=mcmc_burnin,
                       nchain=mcmc_nchain))

### Check 10 fold Cross Validation RMSE

In [9]:

rmse_bma = []

for fold_id in range(1, 11):
    # prepare cross-validation data
    X_te = X_train1[training51.index[training51["fold"] == fold_id]]
    X_tr = X_train1[training51.index[training51["fold"] != fold_id]]
    Y_te = Y_train[training51.index[training51["fold"] == fold_id]]
    Y_tr = Y_train[training51.index[training51["fold"] != fold_id]]

    base_preds_tr=base_preds_train.numpy()[training51.index[training51["fold"] != fold_id]]
    base_preds_te=base_preds_train.numpy()[training51.index[training51["fold"] == fold_id]]

    # build model & run MCMC
    bma_prior, bma_gp_config = bma_dist(X_tr, 
                                    base_preds_tr, 
                                    **bma_model_config)

    bma_model_config.update(bma_gp_config)


    bma_gp_w_samples = run_posterior_inference(model_dist=bma_prior, 
                                           model_config=bma_model_config,
                                           Y=Y_tr, 
                                           map_config=map_config,
                                           mcmc_config=mcmc_config)


    bma_joint_samples = make_bma_samples(X_te, None, base_preds_te, 
                                     bma_weight_samples=bma_gp_w_samples[0],
                                     bma_model_config=bma_model_config,
                                     n_samples=bma_n_samples_eval, 
                                     seed=bne_seed,
                                     y_samples_only=False)

    y_pred = bma_joint_samples['y']
    y_pred = tf.reduce_mean(y_pred, axis=0)

    rmse_bma.append(rmse(Y_te, y_pred))
rmse_bma


/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


Running MAP:	298387.0625...115085.359375...107380.96875...104392.9375...102979.875...100910.546875...100026.5390625...99396.78125...97088.71875...96693.546875...Done.
Running MCMC:	Acceptance Ratio: 0.6906112432479858
Running MAP:	400250.5...147092.484375...132666.96875...125622.5234375...120866.359375...119381.1640625...118420.5078125...116862.796875...116063.5234375...115449.2109375...Done.
Running MCMC:	Acceptance Ratio: 0.6670030951499939
Running MAP:	404433.375...138287.171875...125506.484375...120236.640625...116915.03125...115240.6796875...114216.03125...113518.546875...112961.1171875...112534.890625...Done.
Running MCMC:	Acceptance Ratio: 0.7566648721694946
Running MAP:	362267.125...135549.671875...122294.9765625...118344.421875...116712.0703125...115914.1328125...115478.1171875...115205.0546875...115034.453125...114924.6484375...Done.
Running MCMC:	Acceptance Ratio: 0.7093912959098816
Running MAP:	283982.90625...105229.71875...94491.40625...90671.640625...88751.4765625...87855

In [ ]:
np.mean(rmse_bma)

1.14731

### Build model

In [ ]:
bma_prior, bma_gp_config = bma_dist(X_train1, 
                                    base_preds_train, 
                                    **bma_model_config)

bma_model_config.update(bma_gp_config)

# Check if the model graph is specified correctly.
bma_prior.resolve_graph()

(('gp_weights', ()), ('y', ('gp_weights',)))

### Run MCMC

In [ ]:
# bma_gp_w_samples, chain_samples, sampler_stat = run_posterior_inference(model_dist=bma_prior, 
#                                            model_config=bma_model_config,
#                                            Y=Y_train, 
#                                            map_config=map_config,
#                                            mcmc_config=mcmc_config)

# Above the debug mode

bma_gp_w_samples = run_posterior_inference(model_dist=bma_prior, 
                                           model_config=bma_model_config,
                                           Y=Y_train, 
                                           map_config=map_config,
                                           mcmc_config=mcmc_config)


bma_joint_samples = make_bma_samples(X_test1, None, base_preds_test, 
                                     bma_weight_samples=bma_gp_w_samples[0],
                                     bma_model_config=bma_model_config,
                                     n_samples=bma_n_samples_eval, 
                                     seed=bne_seed,
                                     y_samples_only=False)



Running MAP:	428139.1875...217879.703125...211721.484375...208145.953125...206213.78125...203751.0625...201055.578125...199206.984375...197395.78125...195403.625...Done.
Running MCMC:	Acceptance Ratio: 0.850577712059021


Check with Simple linear regression

In [ ]:
#np.mean((means_pred - means_true)**2) / np.var(means_true)
# mse = tf.reduce_mean((means_train_mcmc-Y_train_mcmc)** 2)/ np.var(Y_train_mcmc)
# mse

from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


def rmse(y_obs, y_pred):
    return np.sqrt(np.mean((y_obs - y_pred) ** 2))


reg = LinearRegression().fit(X_train1, Y_train)
y_pred = reg.predict(X_train1)
rmse_lr = mean_squared_error(y_true=Y_train, y_pred=y_pred, squared=False)
print("Linear Regression RMSE: ", rmse_lr)

Linear Regression RMSE:  1.6748887


In [ ]:
# # Compute sample stats.
# sample_mean = tf.reduce_mean(means_train_mcmc, axis=[0, 1])
# # ==> approx equal to weights

# sample_var = tf.reduce_mean(
#     tf.math.squared_difference(chain_samples, sample_mean),
#     axis=[0, 1])
# ==> less than 1

# def rmse(y_obs, y_pred):
#     """Computes root mean square error."""
#     return np.sqrt(np.mean((y_obs.squeeze() - y_pred.squeeze()) ** 2))
# rmse(Y_train_mcmc, means_train_mcmc)

# samples = samples[:num_sample]
# means_pred = np.mean(samples, axis=0)
# stds_pred = np.std(samples, axis=0)
# mse_ind = np.mean(
#     (means_pred[ind_ids] - means_true[ind_ids])**2) / np.var(means_true[ind_ids])

### Prepare Data for BAE/BNE

In [ ]:
# Construct data from BMA samples, shapes (num_samples * num_data, ...)
means_train_mcmc, X_train_mcmc, Y_train_mcmc = make_bma_samples(
    X_train1, Y_train, base_preds_train, 
    bma_weight_samples=bma_gp_w_samples[0],
    bma_model_config=bma_model_config,
    n_samples=bma_n_samples_train,
    seed=bma_seed, 
    prepare_mcmc_training=True)

# Mean samples based on test data, shape (num_samples, num_data, num_output).
# It is used to generate final examples in `make_bne_samples()`.
means_test_mcmc = make_bma_samples(
    X_test1, None, base_preds_test, 
    bma_weight_samples=bma_gp_w_samples[0],
    bma_model_config=bma_model_config,
    n_samples=bma_n_samples_test,
    seed=bma_seed)

/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


In [ ]:
# spcv_df = tf.repeat(training51["fold"], bma_n_samples_train, axis=0)

# fold_id=2

# X_te = X_train_mcmc.numpy()[spcv_df==fold_id]
# X_tr = X_train_mcmc.numpy()[spcv_df!=fold_id]
# Y_te = Y_train_mcmc[spcv_df==fold_id]
# Y_tr = Y_train_mcmc[spcv_df!=fold_id]
# means_tr_mcmc = means_train_mcmc.numpy()[spcv_df!=fold_id]
# means_te_mcmc = means_train_mcmc.numpy()[spcv_df==fold_id]

# base_preds_tr=base_preds_train.numpy()[training51.index[training51["fold"] != fold_id]]
# base_preds_te=base_preds_train.numpy()[training51.index[training51["fold"] == fold_id]]

# # Construct posterior sampler.
# # Construct posterior sampler.
# bne_prior, bne_gp_config = bne_model_dist(
#     inputs=X_tr,
#     mean_preds=means_tr_mcmc,
#     **bne_model_config)

# bne_model_config.update(bne_gp_config)


# # Estimates GP weight posterior using MCMC.
# bne_gp_w_samples = run_posterior_inference(model_dist=bne_prior,
#                                            model_config=bne_gp_config,
#                                            Y=Y_tr,
#                                            map_config=map_config,
#                                            mcmc_config=mcmc_config,
#                                            initialize_from_map=True)
# # Generates the posterior sample for all model parameters. 
# bne_joint_samples = make_bne_samples(X_te,
#                                      mean_preds=means_te_mcmc,
#                                      bne_model_config=bne_model_config,
#                                      bne_weight_samples=bne_gp_w_samples[0],
#                                      seed=bne_seed)
# y_pred = bne_joint_samples['y']
# means_pred = np.mean(y_pred, axis=0)
# rmse_bne = np.sqrt(np.mean((means_pred - Y_te)**2))
# rmse_bne

## Bayesian Additive Ensemble

Given $\mu(x)$ the posterior of a Bayesian ensemble model, the Bayesian Additive Ensemble is defined as:    

$y \sim N(\mu(x) + r(x), \sigma^2)$

$r \sim GaussianProcess(0, k)$

The additive ensemble $r(x)$ services two purposes: 

1. Mitigates systematic bias in model prediction; 
2. Quantifies the model's epistemic uncertainty.

In [ ]:
# # # BNE GP Configs.
# # lengthscale = 1. # @param
# # l2_regularizer = 10. # @param

# BNE model configs. 
# If estimate_mean=False, only estimates a constant variance on top of the 
# original model.
estimate_mean = "True" # @param ["True", "False"]
variance_prior_mean=0. # @param
# # MAP and MCMC configs
# map_step_size=0.1 # @param
# map_num_steps=10_000 # @param

# mcmc_step_size=1e-2 # @param
# mcmc_num_steps=10_000 # @param

bne_gp_config = DEFAULT_GP_CONFIG.copy()
bne_model_config = DEFAULT_BNE_CONFIG.copy()

map_config = DEFAULT_MAP_CONFIG.copy()
mcmc_config = DEFAULT_MCMC_CONFIG.copy()


bne_gp_config.update(dict(lengthscale=bne_gp_lengthscale, 
                          l2_regularizer=bne_gp_l2_regularizer))
bne_model_config.update(dict(estimate_mean=eval(estimate_mean),
                             variance_prior_mean=variance_prior_mean,
                             **bne_gp_config))

map_config.update(dict(learning_rate=map_step_size,
                       num_steps=map_num_steps))
mcmc_config.update(dict(step_size=mcmc_step_size, 
                        num_steps=mcmc_num_steps,
                       burnin=mcmc_burnin,
                       nchain=mcmc_nchain))

In [ ]:
spcv_df = tf.repeat(training51["fold"], bma_n_samples_train, axis=0)
rmse_bae = []
for fold_id in range(1,11):
    X_te = X_train_mcmc.numpy()[spcv_df==fold_id]
    X_tr = X_train_mcmc.numpy()[spcv_df!=fold_id]
    Y_te = Y_train_mcmc[spcv_df==fold_id]
    Y_tr = Y_train_mcmc[spcv_df!=fold_id]
    means_tr_mcmc = means_train_mcmc.numpy()[spcv_df!=fold_id]
    means_te_mcmc = means_train_mcmc.numpy()[spcv_df==fold_id]

    base_preds_tr=base_preds_train.numpy()[training51.index[training51["fold"] != fold_id]]
    base_preds_te=base_preds_train.numpy()[training51.index[training51["fold"] == fold_id]]

    # Construct posterior sampler.
    bne_prior, bne_gp_config = bne_model_dist(
        inputs=X_tr,
        mean_preds=means_tr_mcmc,
        **bne_model_config)

    bne_model_config.update(bne_gp_config)


    # Estimates GP weight posterior using MCMC.
    bne_gp_w_samples = run_posterior_inference(model_dist=bne_prior,
                                            model_config=bne_gp_config,
                                            Y=Y_tr,
                                            map_config=map_config,
                                            mcmc_config=mcmc_config,
                                            initialize_from_map=True)
    # Generates the posterior sample for all model parameters. 
    bne_joint_samples = make_bne_samples(X_te,
                                        mean_preds=means_te_mcmc,
                                        bne_model_config=bne_model_config,
                                        bne_weight_samples=bne_gp_w_samples[0],
                                        seed=bne_seed)
    
    y_pred = bne_joint_samples['y']
    means_pred = np.mean(y_pred, axis=0)
    rmse_bae.append(np.sqrt(np.mean((means_pred - Y_te)**2)))
print("10 fold cross validation rmse:", rmse_bae)
print("mean rmse:", np.mean(rmse_bae))


Running MAP:	5027.3525390625...4667.4423828125...4667.14892578125...4667.1494140625...4667.1494140625...4667.1494140625...4667.1494140625...4667.1494140625...4667.1494140625...4667.1484375...Done.
Running MCMC:	Acceptance Ratio: 0.6501122713088989
Running MAP:	6941.068359375...5982.5302734375...5979.0703125...5979.0546875...5979.0546875...5979.0546875...5979.0546875...5979.0537109375...5979.0537109375...5979.0537109375...Done.
Running MCMC:	Acceptance Ratio: 0.7149558067321777
Running MAP:	6899.20703125...6056.43359375...6054.19873046875...6054.1953125...6054.19384765625...6054.19482421875...6054.1943359375...6054.19482421875...6054.19482421875...6054.19482421875...Done.
Running MCMC:	Acceptance Ratio: 0.754821240901947
Running MAP:	6668.78857421875...6144.14501953125...6143.5908203125...6143.58984375...6143.5908203125...6143.5908203125...6143.5908203125...6143.5908203125...6143.5908203125...6143.5908203125...Done.
Running MCMC:	Acceptance Ratio: 0.6947862505912781
Running MAP:	5832.90

### Build Model

In [ ]:
# Construct posterior sampler.
bne_prior, bne_gp_config = bne_model_dist(
    inputs=X_train_mcmc,
    mean_preds=means_train_mcmc,
    **bne_model_config)

bne_model_config.update(bne_gp_config)
print(f'prior model graph: {bne_prior.resolve_graph()}')

prior model graph: (('gp_weights', ()), ('y', ('gp_weights',)))


### Run MCMC

In [ ]:
# Estimates GP weight posterior using MCMC.
bne_gp_w_samples, chain_samples, sampler_stat = run_posterior_inference(model_dist=bne_prior,
                                           model_config=bne_gp_config,
                                           Y=Y_train_mcmc,
                                           map_config=map_config,
                                           mcmc_config=mcmc_config,
                                           initialize_from_map=True)
# Generates the posterior sample for all model parameters. 
bne_joint_samples = make_bne_samples(X_test1,
                                     mean_preds=means_test_mcmc,
                                     bne_model_config=bne_model_config,
                                     bne_weight_samples=bne_gp_w_samples[0],
                                     seed=bne_seed)

Running MAP:	8040.71630859375...7257.259765625...7244.951171875...7239.53125...7238.07568359375...7237.91650390625...7237.91259765625...7237.912109375...7237.912109375...7237.912109375...Done.
Running MCMC:	Acceptance Ratio: 0.7726417183876038


In [ ]:
surface_pred_bae = {k: np.mean(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}
surface_var_bae = {k: np.var(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}

# dealing with NA

In [ ]:
# percentile of NAN
np.sum(np.isnan(np.mean(bne_joint_samples['mean_original'], axis=0)))/84421

0.005235664112010045

## Bayesian Nonparametric Ensemble (Variance Only)
So far, we are only estimating the mean-component of the model, i.e., we are assuming: 

$y \sim Gaussian(m(x), \sigma^2); \quad m(x) = GP(0, k)$.

By doing so, the model is implicitly assuming the distribution of $y$ is always a symmetric Gaussian distribution with constant mean across space and time. As a result, our model can only quantify model uncertainty (due to lack of data) via the GP prior, but cannot flexibly capture the data uncertainty that is inherent to the empirical distribution of y.

To resolve this, we extend the ensemble's outcome distribution $y | f$ by also estimating the higher moments of the data distribution (e.g., variance, skewness, etc) using flexible estimators. Specifically, we specify the outcome distribution family to the [maximum-entropy distribution](https://en.wikipedia.org/wiki/Principle_of_maximum_entropy) given the known moments, so the predictive distribution is [minimax](https://arxiv.org/pdf/math/0410076.pdf) and still statistically efficient to estimate.

For example, when we want to estimate the first two moments (mean and variance) of the distribution, this leads to a Gaussian distribution with spatio-temporally adaptive variance $\sigma(x)^2$:

$$y \sim Gaussian(m(x), \sigma(x)^2); \quad \mbox{where} \quad m \sim GP(0, k_m), \sigma \sim GP(0, k_\sigma)$$

and when we want to estimate the first three moments (mean and variance) of the distribution, this leads to a [Exponentially-modifed Gaussian](https://en.wikipedia.org/wiki/Exponentially_modified_Gaussian_distribution) (EMG) distribution with spatio-temporally adaptive variance $\sigma(x)^2$ and skewness $\lambda(x)$:

$$y \sim EMG(m(x), \sigma(x)^2, \lambda(x)); \quad \mbox{where} \quad m \sim GP(0, k_m), \sigma \sim GP(0, k_\sigma), \lambda \sim GP(0, k_\lambda)$$




### Define Model & Run MCMC

In [ ]:
# Construct posterior sampler.
bne_prior, bne_gp_config = bne_model_dist(
    inputs=X_train_mcmc,
    mean_preds=means_train_mcmc,
    **bne_model_config)

bne_model_config.update(bne_gp_config)
print(f'prior model graph: {bne_prior.resolve_graph()}')

# Estimates GP weight posterior using MCMC.
bne_gp_w_samples = run_posterior_inference(model_dist=bne_prior,
                                           model_config=bne_gp_config,
                                           Y=Y_train_mcmc,
                                           map_config=map_config,
                                           mcmc_config=mcmc_config,
                                           initialize_from_map=True)
# Generates the posterior sample for all model parameters. 
bne_joint_samples = make_bne_samples(X_test1,
                                     mean_preds=means_test_mcmc,
                                     bne_model_config=bne_model_config,
                                     bne_weight_samples=bne_gp_w_samples[0],
                                     seed=bne_seed)

prior model graph: (('gp_weights', ()), ('y', ('gp_weights',)))
Running MAP:	7446.732421875...7053.1435546875...7051.3798828125...7051.36865234375...7051.36865234375...7051.36865234375...7051.36865234375...7051.36865234375...7051.36865234375...7051.36865234375...Done.
Running MCMC:	Acceptance Ratio: 0.6818126440048218


In [ ]:
surface_pred_bne_vo = {k: np.mean(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}
surface_var_bne_vo = {k: np.var(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}



## Bayesian Nonparametric Ensemble (Variance + Skewness)

### Define Model & Run MCMC

In [ ]:
# Construct prior distribution.
bne_prior, bne_gp_config = bne_model_dist(
    inputs=X_train_mcmc,
    mean_preds=means_train_mcmc,
    **bne_model_config)

bne_model_config.update(bne_gp_config)
print(f'prior model graph: {bne_prior.resolve_graph()}')
# Estimates GP weight posterior using MCMC.
bne_gp_w_samples = run_posterior_inference(model_dist=bne_prior,
                                           model_config=bne_gp_config,
                                           Y=Y_train_mcmc,
                                           map_config=map_config,
                                           mcmc_config=mcmc_config,
                                           initialize_from_map=True)
# Generates the posterior sample for all model parameters. 
bne_joint_samples = make_bne_samples(X_test1,
                                     mean_preds=means_test_mcmc,
                                     bne_model_config=bne_model_config,
                                     bne_weight_samples=bne_gp_w_samples[0],
                                     seed=bne_seed)

prior model graph: (('gp_weights', ()), ('y', ('gp_weights',)))
Running MAP:	7174.9990234375...7055.798828125...7053.3818359375...7053.35888671875...7053.3583984375...7053.35888671875...7053.35791015625...7053.35791015625...7053.35791015625...7053.3583984375...Done.
Running MCMC:	Acceptance Ratio: 0.7549411058425903


In [ ]:
surface_pred_bne_vs = {k: np.mean(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}
surface_var_bne_vs = {k: np.var(np.nan_to_num(bne_joint_samples[k]), axis=0) for k in ('y', 'mean_original', 'resid')}

## Basic Plots

In [ ]:
#_DATA_ADDR_PREFIX = "./example/data"
# Tuning Parameters
BMA_lenthscale = bma_gp_lengthscale
BNE_lenthscale = bne_gp_lengthscale
BMA_L2 = bma_gp_l2_regularizer
BNE_L2 = bne_gp_l2_regularizer
_SAVE_ADDR_PREFIX = "./pic/BMA_lenthscale_{}_L2_{}_BNE_lenthscale_{}_L2_{}".format(BMA_lenthscale, BMA_L2, BNE_lenthscale, BNE_L2)

path=_SAVE_ADDR_PREFIX
isExists=os.path.exists(path) #判断路径是否存在，存在则返回true

if not isExists:
    os.makedirs(path)


### 1. The predictive surface of individual base models.

In [ ]:
coordinate = np.asarray(base_model_predictions_eastMA[["lon", "lat"]].values.tolist()).astype(np.float32)
monitors = np.asarray(training_eastMA_noMI[["lon", "lat"]].values.tolist()).astype(np.float32)
base_model_names = ["pred_av", "pred_gs", "pred_caces"]

base_model_predictions_eastMA[["pred_av", "pred_gs", "pred_caces"]] = np.where(np.isnan(base_model_predictions_eastMA[["pred_av", "pred_gs", "pred_caces"]]), 0, base_model_predictions_eastMA[["pred_av", "pred_gs", "pred_caces"]])
color_norm_base = make_color_norm(
    base_model_predictions_eastMA[["pred_av", "pred_gs", "pred_caces"]],   
    method="percentile")

for base_model_name in base_model_names:
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                             'base_model_{}_bmals_{}_r_{}_bnels_{}_r_{}.png'.format(
                                 base_model_name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
                                 bne_gp_lengthscale, bne_gp_l2_regularizer))
    
    posterior_heatmap_2d(base_model_predictions_eastMA[base_model_name], coordinate,
                         monitors,
                         cmap='RdYlGn_r',
                         norm=color_norm_base, 
                         #norm_method="percentile",
                         save_addr=save_name)

### 2. The predictive surface of individual BNE gp weights

In [ ]:
bma_ensemble_weights = bma_joint_samples['ensemble_weights']
ensemble_weights_val = tf.reduce_mean(bma_ensemble_weights, axis=0)

weights_dict = {
    "AV": ensemble_weights_val[:, 0],
    "GS": ensemble_weights_val[:,1],
    "CACES": ensemble_weights_val[:,2],
}
#weights_dict
color_norm_weights = make_color_norm(
    list(weights_dict.values()),#[2],   
    method="percentile")

In [ ]:
ensemble_weights_var = np.var(bma_ensemble_weights, axis=0)
weights_var_dict = {
    "AV": ensemble_weights_var[:, 0],
    "GS": ensemble_weights_var[:,1],
    "CACES": ensemble_weights_var[:,2],
}
#weights_dict
color_norm_weights_var = make_color_norm(
    list(weights_var_dict.values()),#[0],   
    method="percentile")
# display(ensemble_weights_val,ensemble_weights_var)

In [ ]:
base_model_names = ["AV", "GS", "CACES"]
for base_model_name in base_model_names:
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                             'base_weights_{}_bmals_{}_r_{}_bnels_{}_r_{}.png'.format(
                                 base_model_name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
                                 bne_gp_lengthscale, bne_gp_l2_regularizer))
    
    posterior_heatmap_2d(weights_dict[base_model_name], coordinate,
                         monitors,
                         cmap='viridis',
                         norm=color_norm_weights, 
                         #norm_method="percentile",
                         #save_addr='')
                         save_addr=save_name)

In [ ]:
# plot weights' variance
for base_model_name in base_model_names:
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                             'base_wvar_{}_bmals_{}_r_{}_bnels_{}_r_{}.png'.format(
                                 base_model_name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
                                 bne_gp_lengthscale, bne_gp_l2_regularizer))
    
    posterior_heatmap_2d(weights_var_dict[base_model_name], coordinate,
                         monitors,
                         cmap='viridis',
                         norm=color_norm_weights_var, 
                         #norm_method="percentile",
                         #save_addr='')
                         save_addr=save_name)

### 3. The predictive surface of Y_mean, residual process, and Y_mean + residual process.

In [ ]:
# BAE
color_norm_pred = make_color_norm(
    #np.nan_to_num(list(surface_pred_bae.values())[:2][0]),
    list(surface_pred_bae.values())[:2],  
    method="percentile")

color_norm_pred_r = make_color_norm(
    #np.nan_to_num(list(surface_pred_bae.values())[2:]),
    list(surface_pred_bae.values())[2],  
    method="residual_percentile")


for name, value in surface_pred_bae.items():
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                            'BAE_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
                                name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
                                bne_gp_lengthscale, bne_gp_l2_regularizer))

    value = np.where(np.isnan(value), 0, value)
    color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
                                                  cmap='RdYlGn_r',
                    norm= color_norm_pred_r if name=='resid' else color_norm_pred,
                    #norm_method="percentile",
                    #save_addr='')
                    save_addr=save_name)
    

In [ ]:
# BNE vo
color_norm_pred = make_color_norm(
    #np.nan_to_num(list(surface_pred_bae.values())[:2][0]),
    list(surface_pred_bne_vo.values())[:2],  
    method="percentile")

color_norm_pred_r = make_color_norm(
    #np.nan_to_num(list(surface_pred_bae.values())[2:]),
    list(surface_pred_bne_vo.values())[2],  
    method="residual_percentile")


for name, value in surface_pred_bne_vo.items():
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                            'BNEvo_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
                                name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
                                bne_gp_lengthscale, bne_gp_l2_regularizer))

    value = np.where(np.isnan(value), 0, value)
    color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
                                                  cmap='RdYlGn_r',
                    norm= color_norm_pred_r if name=='resid' else color_norm_pred,
                                      save_addr=save_name)
                    #norm_method="percentile",
                    #save_addr='')
    

In [ ]:
# BNE v+s
color_norm_pred = make_color_norm(
    #np.nan_to_num(list(surface_pred_bae.values())[:2][0]),
    list(surface_pred_bne_vs.values())[:2],  
    method="percentile")

color_norm_pred_r = make_color_norm(
    #np.nan_to_num(list(surface_pred_bae.values())[2:]),
    list(surface_pred_bne_vs.values())[2],  
    method="residual_percentile")


for name, value in surface_pred_bne_vs.items():
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                            'BNEvs_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(name, bma_gp_lengthscale, 
                                bma_gp_l2_regularizer, bne_gp_lengthscale, bne_gp_l2_regularizer))

    value = np.where(np.isnan(value), 0, value)
    color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
                                                  cmap='RdYlGn_r',
                    norm= color_norm_pred_r if name=='resid' else color_norm_pred,
                    #norm_method="percentile",
                    #save_addr='')
                save_addr=save_name)
    

### 4.The predictive variance of Y_mean, residual process, and Y.

In [ ]:
# BAE
color_norm_var = make_color_norm(
    list(surface_var_bae.values())[:2], 
    method="percentile")

color_norm_var_r = make_color_norm(
    list(surface_var_bae.values())[2], 
    method="percentile")


for name, value in surface_var_bae.items():
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                            'var_BAE_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
                                name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
                                bne_gp_lengthscale, bne_gp_l2_regularizer))
    #value = np.where(np.isnan(value), 0, value)
    color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
                                cmap='inferno_r',
                                norm= color_norm_var_r if name=='resid' else color_norm_var,
                                #norm_method="percentile",
                                save_addr=save_name)

In [ ]:
# BNE vo
color_norm_var = make_color_norm(
    list(surface_var_bne_vo.values())[:2], 
    method="percentile")

color_norm_var_r = make_color_norm(
    list(surface_var_bne_vo.values())[2], 
    method="percentile")


for name, value in surface_var_bne_vo.items():
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                            'var_BNEvo_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
                                name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
                                bne_gp_lengthscale, bne_gp_l2_regularizer))
    #value = np.where(np.isnan(value), 0, value)
    color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
                                cmap='inferno_r',
                                norm= color_norm_var_r if name=='resid' else color_norm_var,
                                #norm_method="percentile",
                                save_addr=save_name)

In [ ]:
# BNE v+s
color_norm_var = make_color_norm(
    list(surface_var_bne_vs.values())[:2], 
    method="percentile")

color_norm_var_r = make_color_norm(
    list(surface_var_bne_vs.values())[2], 
    method="percentile")


for name, value in surface_var_bne_vs.items():
    save_name = os.path.join(_SAVE_ADDR_PREFIX,
                            'var_BNEvs_{}_bma:ls_{}_r_{}_bne:ls_{}_r_{}.png'.format(
                                name, bma_gp_lengthscale,  bma_gp_l2_regularizer,
                                bne_gp_lengthscale, bne_gp_l2_regularizer))
    #value = np.where(np.isnan(value), 0, value)
    color_norm = posterior_heatmap_2d(value, X=coordinate, X_monitor=monitors,
                                cmap='inferno_r',
                                norm= color_norm_var_r if name=='resid' else color_norm_var,
                                #norm_method="percentile",
                                save_addr=save_name)

### Evaluation

In [ ]:
# @title Simulation: compute_metrics
def compute_metrics(data_dict, q_true=None, ind_ids=None, num_sample=None):
  if q_true is None:
    q_true = np.array(
        [0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25,
         0.75, 0.8, 0.85, 0.9, 0.925, 0.95, 0.975])

  if ind_ids is None:
    # Find IDs of in-domain test data via range comparison 
    # between X_train and X_test.
    X_train_min = np.min(data_dict['X_train'], axis=0)
    X_train_max = np.max(data_dict['X_train'], axis=0)

    test_ids_greater_than_min = np.all(
        data_dict['X_test'] > X_train_min, axis=-1)
    test_ids_less_than_max = np.all(
        data_dict['X_test'] < X_train_max, axis=-1)

    ind_ids = np.where(
        np.logical_and(test_ids_greater_than_min, test_ids_less_than_max))[0]

  samples = data_dict[f'{model_name}_samples']
  means_true = data_dict['mean_test']
  y_test = data_dict['Y_test']

  if num_sample is not None:
    samples = samples[:num_sample]

  means_pred = np.mean(samples, axis=0)
  stds_pred = np.std(samples, axis=0)
  quantile_pred = np.quantile(samples, q=q_true, axis=0)

  # Compute in-domain metrics.
  nll_ind = np.mean(
      ((means_pred[ind_ids] - means_true[ind_ids])/stds_pred[ind_ids])**2 + 
      np.log(stds_pred[ind_ids]))
  clb_ind = np.mean(
      ((means_pred[ind_ids] - means_true[ind_ids])/stds_pred[ind_ids])**2)
  shp_ind = np.mean(np.log(stds_pred[ind_ids]))
  mse_ind = np.mean(
      (means_pred[ind_ids] - means_true[ind_ids])**2) / np.var(means_true[ind_ids])

  q_pred_ind = np.mean(y_test[ind_ids] < quantile_pred[:, ind_ids], axis=(1, 2))
  ece_ind = np.mean((q_pred_ind - q_true)**2)
  cov_prob_95_ind = q_pred_ind[-1] - q_pred_ind[0]
  cov_prob_90_ind = q_pred_ind[-2] - q_pred_ind[1]
  cov_prob_85_ind = q_pred_ind[-3] - q_pred_ind[2]
  cov_prob_80_ind = q_pred_ind[-4] - q_pred_ind[3]

  # Compute all-domain (ind + ood) metrics.
  nll_all = np.mean(((means_pred - means_true)/stds_pred)**2 + np.log(stds_pred))
  clb_all = np.mean(((means_pred - means_true)/stds_pred)**2)
  shp_all = np.mean(np.log(stds_pred))
  mse_all = np.mean((means_pred - means_true)**2) / np.var(means_true)

  q_pred_all = np.mean(y_test < quantile_pred, axis=(1, 2))
  ece_all = np.mean((q_pred_all - q_true)**2)
  cov_prob_95_all = q_pred_all[-1] - q_pred_all[0]
  cov_prob_90_all = q_pred_all[-2] - q_pred_all[1]
  cov_prob_85_all = q_pred_all[-3] - q_pred_all[2]
  cov_prob_80_all = q_pred_all[-4] - q_pred_all[3]

  return (mse_ind, nll_ind, clb_ind, shp_ind, ece_ind, cov_prob_95_ind, cov_prob_90_ind, cov_prob_85_ind, cov_prob_80_ind,
          mse_all, nll_all, clb_all, shp_all, ece_all, cov_prob_95_all, cov_prob_90_all, cov_prob_85_all, cov_prob_80_all)